# Chapter 4: Basic properties of feedback

## Example 4.6: PID_Control_of_DC_Motor_Speed.sce

In [ ]:
//Example 4.6 
//PID Control of DC Motor Speed.

//------------------------------------------------------------------
//NOTE THAT--

//The model as given in matlab program for this example in the book is

//num=Ra*s + La*s^2 ;
//den=Ke*ki + (Ra*Ke*Ke+Ke*kp)*s + (Ra*b+Ke*Ke+Ke*kd)*s^2 + Jm*La*s^3;

//this does not match to the model of DC motor given on page 43. 
//Also, if we assume this model, disturbance response given 
//in figure 4.13 (a) 
//is different from expected.
//For instance, with P control, output should asymptotically go to 0
//for disturbance step input, because numerator is s(Ra + La*s) 
//and system is type 0 (no pole at origin).
//i.e. y(inf)=lim s->0 s*Y(s)= s*[s(Ra + La*s)/den]*1/s=0;

//In following code, we have considered correct model of DC motor as 
//given on page 43. Note that, this model must have been used 
//by authors of the book for 
//step reference tracking as it is correctly shown in figure 4.13 (b)

//------------------------------------------------------------------
xdel(winsid())//close all graphics Windows
clear;
clc;

//------------------------------------------------------------------
// System parameters
Jm=0.0113; // N-m-s^2/rad
b=0.028;   // N-m-s/rad
La=0.1;    // henry
Ra=0.45;   // ohms
Kt=0.067   // n-m/amp
Ke=0.067;  // V-sec/amp

// Controller parameters
kp=3;
ki=15;    // sec^-1
kd=0.3;   // sec

// DC Motor Transfer function as given on page 43 of book (edition 5)
//G=Kt/[Jm*La s^2 + (Jm*Ra + La*b)s +(Ra*b +Kt*Ke)]
s=%s;
num=[Kt];
den=[(Ra*b +Kt*Ke) (Jm*Ra + La*b) Jm*La];
Ns=poly(num,'s','coeff');
Ds=poly(den,'s','coeff');
G=syslin('c',Ns/Ds)

//PID controller, Gc=(kd s^2 + kp s + ki)/s
num=[ki kp kd;ki kp 0;0 kp 0]; //numerator parameters of controller)
                                 //(row wise for PID, PI and P)
den=[0 1];                    //denominator parameters of controller
Ds=poly(den,'s','coeff');    //denominator polynomial of controller
t=0:0.005:10;               // Simulation time
//------------------------------------------------------------------
//Step disturbance response with P, PI and PID controller. 

for i=1:3
Ns=poly(num(i,:),'s','coeff');//numerator polynomial of controller
sysG=syslin('c',Ns/Ds);
sysD=G/. sysG;
v(i,:)=csim('step',t,sysD);
end
plot(t',v');
//Title, labels and grid to the figure
exec .ig_settings.sci; //custom script to set the figure properties
title('Responses of P,PI and PID control to step disturbance...
 input','fontsize',3)
xlabel('Time t (sec.)','fontsize',2)
ylabel('Amplitude','fontsize',2)
hl=legend(['PID','PI','P']);

//------------------------------------------------------------------
//Reference step response

figure
for i=1:3
Ns=poly(num(i,:),'s','coeff');
Gc=syslin('c',Ns/Ds);
// Step reference response with P, PI and PID controller.
sysR=G*Gc/(1+G*Gc);
v(i,:)=csim('step',t,sysR);
end
plot(t',v')
//Title, labels and grid to the figure
exec .ig_settings.sci; //custom script to set the figure properties
title('Responses of PID control to step reference input','fontsize',3)
xlabel('Time t (sec.)','fontsize',2)
ylabel('Amplitude','fontsize',2)
hl=legend(['PID','PI','P']);

//------------------------------------------------------------------

## Example 4.7: Discrete_Equivalent.sce

In [ ]:
//Example 4.7
//Discrete Equivalent.
//------------------------------------------------------------------
xdel(winsid())//close all graphics Windows
clear;
clc;

// Transfer function
s=%s; 
num=[1 11];
den=[1 3]
Us=poly(num,'s','coeff');
Es=poly(den,'s','coeff');
Ds=syslin('c',Us/Es);
sysc=tf2ss(Ds)

//Discretize the system using sampling time Ts=1 and Bilinear Transform
Ts=1;
sysd=cls2dls(sysc,Ts);

//Pulse transfer function
Dd=ss2tf(sysd)
disp(Dd,'Dd=')
disp('Note that, multiply numerator and denomintor each by 7... 
 will give the result as in book.')
//------------------------------------------------------------------

## Example 4.8: Equivalent_discrete_controller_for_DC_motor_speed_control.sce

In [ ]:
//Example 4.8
//Equivalent discrete controller for DC motor speed control.
//------------------------------------------------------------------
//NOTE THAT-- The system response (continuous) to sampled control 
//input depends on 
//the sampling time set for continuous signal in SIMULATION.
//In this example we consider sampling period of 0.009 sec 
//to represent continuous time signal.
//------------------------------------------------------------------
xdel(winsid())//close all graphics Windows
clear;
clc;
//------------------------------------------------------------------
// Continuous time system and controller
// System transfer function
s=%s; 
num=[45 0];
den=[45 14 1]
Nms=poly(num,'s','coeff');
Dns=poly(den,'s','coeff');
Gp=syslin('c',Nms/Dns); //system transfer function
// Controller
numDa=[6 1];
denDa=[0 1]
Nms=poly(numDa,'s','coeff');
Dns=poly(denDa,'s','coeff');
sysD=syslin('c',1.4*Nms/Dns); //controller transfer function
//Closed loop responses
num=[1 0];
den=[1 0];
Nms=poly(num,'s','coeff');
Dns=poly(den,'s','coeff');
H=syslin('c',Nms/Dns)
sysDa=Gp*sysD/.H;
//step response and control input
t=0:0.009:5;
yt=csim('step',t,sysDa); //step response
figure(0)
plot2d(t,yt,1)
Gu=sysD/(1+Gp*sysD);
ut=csim('step',t,Gu); //control input
figure(1)
plot2d(t,ut,1)
//------------------------------------------------------------------
sys=tf2ss(Gp); //state space model of the system
con=tf2ss(sysD); //controller state space model
// discrete-time time system and controller
//Discretize the system and control with sampling time Ts=0.07
// using Bilinear Transform
Ts=0.07;
sysDd=cls2dls(sys,Ts); // discrete-time system state space model
conDd=cls2dls(con,Ts); // discrete-time controller state space model
//Pulse transfer function of system
Gpz=ss2tf(sysDd);
//Pulse transfer function of controller
Gcz=ss2tf(conDd);
//Closed loop response
Gz=Gpz*Gcz/(1+Gpz*Gcz)
//Control input pulse transfer function
Guz=Gcz/(1+Gpz*Gcz)
T=0:Ts:5;
r=ones(1,length(T));
yd=flts(r,Gz);............//Discrete respnse to discrete input
ud=flts(r,Guz);           //Discrete Control input
//continuous response for digital input
t=0:0.009:5;
k=0;
for i=1:length(yd)
    for j=1:8
       if (k+j)>length(t) then
          break
          else
          YD(1,k+j)=yd(i);
       end
       end
    k=k+j;
end
yt=csim(1-YD,t,Gp*sysD);
scf(0)
plot2d(t,yt,5);
scf(1)
plot2d2(T,ud,5);
//------------------------------------------------------------------
//Discretize the system and control with sampling time Ts=0.035
// using Bilinear Transform
Ts=0.035;
sysDd=cls2dls(sys,Ts); // discrete-time system state space model
conDd=cls2dls(con,Ts); // discrete-time controller state space model
Gpz=ss2tf(sysDd);      //Pulse transfer function of system
Gcz=ss2tf(conDd);     //Pulse transfer function of controller
//Closed loop response
Gz=Gpz*Gcz/(1+Gpz*Gcz)
//Control input pulse transfer function
Guz=Gcz/(1+Gpz*Gcz)
T=0:Ts:5;
r=ones(1,length(T));
yd=flts(r,Gz);............//Discrete respnse to discrete input
ud=flts(r,Guz);           //Discrete Control input
t=0:0.009:5;
k=0;
for i=1:length(yd)
    for j=1:4
       if (k+j)>length(t) then
          break
          else
          YD(1,k+j)=yd(i);
       end
       end
    k=k+j;
end
yt=csim(1-YD,t,Gp*sysD);
scf(0)
plot2d(t,yt,2);
scf(1)
plot2d2(T,ud,2);
scf(0)
//Title, labels and grid to the figure
exec .ig_settings.sci; //custom script to set the figure properties
title('Comparision plots of Speed-control system with continuous...
 and discrete controllers','fontsize',3)
xlabel('Time t (sec.)','fontsize',2)
hl=legend(['Continuous time','Discrete-time, Ts=0.07 s'...
,'Discrete-time, Ts=0.035 s'],4);
scf(1)
//Title, labels and grid to the figure
exec .ig_settings.sci; //custom script to set the figure properties
title('Comparision plots of Speed-control system with continuous...
 and discrete controllers','fontsize',3)
xlabel('Time t (sec.)','fontsize',2)
hl=legend(['Continuous time','Discrete-time, Ts=0.07 s',...
'Discrete-time, Ts=0.035 s']);
//------------------------------------------------------------------